In [1]:
# 데이터 전처리 

import os
import pandas as pd
import geopandas as gpd

def try_encodings(file_path, encodings=None):
    if encodings is None:
        encodings = ["utf-8", "cp949", "euc-kr"]
        
    for enc in encodings:
        try:
            with open(file_path, "r", encoding=enc) as f:
                f.read()
            return enc
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"다른 예외 발생: {e}")
            continue
    raise ValueError(f"파일을 열 수 있는 인코딩을 찾을 수 없습니다: {file_path}")

filenames = {
    #"life_mobility":   "생활이동 데이터(250m).csv",
    "sports_facility": "체육시설 좌표.csv",
    "trash_bin_2024":  "2024_서울시_쓰레기통_좌표.csv",
    "bus_stops":       "서울시 버스정류소 위치정보.csv",
    "parking_lot":     "주차장 좌표.csv",
    "legal_boundary":  "13.서울시_법정경계(시군구).geojson",
    "school":          "서울시_학교_좌표.csv",
    "sidewalk":        "인도4.csv",
    "road":            "24.서울시_도로명주소(도로).geojson"
}

data_path = "./data"   # 데이터가 저장된 폴더 경로
df = {}

for key, filename in filenames.items():
    full_path = os.path.join(data_path, filename)
    if filename.endswith(".csv"):
        enc = try_encodings(full_path)
        # CSV 파일은 pandas로 불러오기
        df[key] = pd.read_csv(full_path,encoding = enc)
    elif filename.endswith(".geojson"):
        # GeoJSON 파일은 geopandas로 불러오기
        df[key] = gpd.read_file(full_path)
    else:
        # 기타 확장자에 대한 처리를 추가적으로 할 수 있음
        print(f"Unsupported file format for {key}: {filename}")


In [2]:
# 각 데이터프레임의 컬럼 이름을 출력하는 함수
def print_column_names(dataframes):
    """
    딕셔너리에 저장된 데이터프레임들의 컬럼 이름을 출력합니다.
    """
    for key, dataframe in dataframes.items():
        print(f"Dataset '{key}'의 컬럼 이름:")
        try:
            print(dataframe.columns.tolist())
        except AttributeError:
            print("해당 데이터는 데이터프레임 형식이 아닙니다.")
        print("-" * 40)

In [4]:
from shapely.geometry import Point

def create_geodataframe(df, lon_col="longitude", lat_col="latitude", crs="EPSG:4326"):
    """
    Pandas DataFrame에서 GeoPandas GeoDataFrame을 생성하는 함수.
    - 해당 컬럼이 없으면 원본 DataFrame을 그대로 반환합니다.
    
    Parameters:
    - df: 원본 DataFrame
    - lon_col: 경도 컬럼 이름
    - lat_col: 위도 컬럼 이름
    - crs: 좌표계 (기본값: WGS84, "EPSG:4326")
    
    Returns:
    - GeoDataFrame 또는 원본 DataFrame
    """
    if lon_col in df.columns and lat_col in df.columns:
        # Point 객체로 geometry 생성
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        # GeoDataFrame 생성
        gdf = gpd.GeoDataFrame(df, geometry=geometry)
        gdf.set_crs(crs, inplace=True)  # 좌표계 설정
        return gdf
    else:
        # 경도, 위도 컬럼이 없으면 원본 DataFrame 반환
        print(f"경고: '{lon_col}' 또는 '{lat_col}' 컬럼이 존재하지 않습니다. 변환 생략.")
        return df
    


# df 딕셔너리 내 모든 데이터프레임 처리
for key, dataframe in df.items():
    print(f"Processing dataset: {key}")
    df[key] = create_geodataframe(dataframe)



Processing dataset: sports_facility
Processing dataset: trash_bin_2024
Processing dataset: bus_stops
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: parking_lot
Processing dataset: legal_boundary
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: school
Processing dataset: sidewalk
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: road
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.


In [5]:
def convert_to_geodataframe(df, lon_col, lat_col, crs="EPSG:4326"):
    """
    위도와 경도 데이터를 기반으로 Pandas DataFrame을 GeoDataFrame으로 변환합니다.
    
    Parameters:
    - df: Pandas DataFrame
    - lon_col: 경도 컬럼 이름
    - lat_col: 위도 컬럼 이름
    - crs: 좌표계 (기본값: EPSG:4326)
    
    Returns:
    - GeoDataFrame
    """
    # Point 객체로 geometry 생성
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    # GeoDataFrame 생성
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    gdf.set_crs(crs, inplace=True)  # 좌표계 설정
    return gdf
df['bus_stops'] = convert_to_geodataframe(df['bus_stops'],lon_col = 'X좌표',lat_col = 'Y좌표')
df['bus_stops'].head()

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입,geometry
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로,POINT (126.98775 37.56981)
1,100000002,1002,창경궁.서울대학교병원,126.996522,37.579433,중앙차로,POINT (126.99652 37.57943)
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로,POINT (126.99825 37.58258)
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로,POINT (126.98761 37.56858)
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로,POINT (127.00174 37.58624)


In [6]:
re_cols = {'X좌표':'longitude','Y좌표':'latitude'}
df['bus_stops'].rename(columns = re_cols,inplace = True)
df['bus_stops'].head()

,노드 ID,정류소번호,정류소명,longitude,latitude,정류소 타입,geometry
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로,POINT (126.98775 37.56981)
1,100000002,1002,창경궁.서울대학교병원,126.996522,37.579433,중앙차로,POINT (126.99652 37.57943)
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로,POINT (126.99825 37.58258)
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로,POINT (126.98761 37.56858)
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로,POINT (127.00174 37.58624)


In [7]:
print_column_names(df)
df['sports_facility'].head()

Dataset 'sports_facility'의 컬럼 이름:
['Unnamed: 0', '시설주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'trash_bin_2024'의 컬럼 이름:
['Unnamed: 0', 'address', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'bus_stops'의 컬럼 이름:
['노드 ID', '정류소번호', '정류소명', 'longitude', 'latitude', '정류소 타입', 'geometry']
----------------------------------------
Dataset 'parking_lot'의 컬럼 이름:
['Unnamed: 0', '주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'legal_boundary'의 컬럼 이름:
['SIG_CD', 'SIG_KOR_NM', 'geometry']
----------------------------------------
Dataset 'school'의 컬럼 이름:
['학교명', '도로명주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'sidewalk'의 컬럼 이름:
['UFID', 'WIDT', 'QUAL', 'BYYN', 'KIND', 'SCLS', 'FMTA', 'geometry_type', 'coordinates']
----------------------------------------
Dataset 'road'의 컬럼 이름:
['BSI_INT', 'RBP_CN', 'RDS_DPN_SE', 'RDS_MAN_

,Unnamed: 0,시설주소,latitude,longitude,geometry
0,0,서울특별시 구로구 오류로 36-25,37.488639,126.841743,POINT (126.84174 37.48864)
1,1,서울특별시 강서구 공항대로36길 74,37.554982,126.836344,POINT (126.83634 37.55498)
2,2,서울특별시 강서구 양천로61길 101,37.559960,126.865560,POINT (126.86556 37.55996)
3,3,서울특별시 강서구 양천로61길 101,37.559960,126.865560,POINT (126.86556 37.55996)
4,4,서울특별시 강서구 양천로61길 101,37.559960,126.865560,POINT (126.86556 37.55996)


In [ ]:
# 쓸모 없는 칼럼 드랍.
drop_cols = 'Unnamed: 0'
for key, dataframe in df.items():
    if drop_cols in dataframe.columns:
        dataframe.drop(columns=drop_cols, inplace=True)

print_column_names(df)


Dataset 'sports_facility'의 컬럼 이름:
['시설주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'trash_bin_2024'의 컬럼 이름:
['address', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'bus_stops'의 컬럼 이름:
['노드 ID', '정류소번호', '정류소명', 'longitude', 'latitude', '정류소 타입', 'geometry']
----------------------------------------
Dataset 'parking_lot'의 컬럼 이름:
['주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'legal_boundary'의 컬럼 이름:
['SIG_CD', 'SIG_KOR_NM', 'geometry']
----------------------------------------
Dataset 'school'의 컬럼 이름:
['학교명', '도로명주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'sidewalk'의 컬럼 이름:
['UFID', 'WIDT', 'QUAL', 'BYYN', 'KIND', 'SCLS', 'FMTA', 'geometry_type', 'coordinates']
----------------------------------------
Dataset 'road'의 컬럼 이름:
['BSI_INT', 'RBP_CN', 'RDS_DPN_SE', 'RDS_MAN_NO', 'REP_CN', 'RN', 'RN_CD', 'ROAD_BT', '

In [9]:
import os
import geopandas as gpd

output_path = "./processed_geojson_data"  # 저장할 폴더 경로
os.makedirs(output_path, exist_ok=True)  # 폴더가 없으면 생성

for key, dataframe in df.items():
    # GeoDataFrame인지 확인
    if isinstance(dataframe, gpd.GeoDataFrame):
        output_file = os.path.join(output_path, f"{key}.geojson")
        dataframe.to_file(output_file, driver="GeoJSON")
        print(f"{key} 저장 완료: {output_file}")
    else:
        print(f"{key}는 GeoDataFrame이 아닙니다. 저장 건너뜀.")


sports_facility 저장 완료: ./processed_geojson_data\sports_facility.geojson
trash_bin_2024 저장 완료: ./processed_geojson_data\trash_bin_2024.geojson
bus_stops 저장 완료: ./processed_geojson_data\bus_stops.geojson
parking_lot 저장 완료: ./processed_geojson_data\parking_lot.geojson
legal_boundary 저장 완료: ./processed_geojson_data\legal_boundary.geojson
school 저장 완료: ./processed_geojson_data\school.geojson
sidewalk는 GeoDataFrame이 아닙니다. 저장 건너뜀.
road 저장 완료: ./processed_geojson_data\road.geojson


In [16]:
# df['sidewalk'].head()

import geopandas as gpd
from shapely.geometry import shape
import json

def convert_to_geodataframe_from_columns(df, geom_type_col="geometry_type", coords_col="coordinates", crs="EPSG:4326"):
    """
    geometry_type과 coordinates 컬럼을 기반으로 GeoDataFrame을 생성합니다.
    
    Parameters:
    - df: Pandas DataFrame
    - geom_type_col: 지오메트리 타입을 나타내는 컬럼 이름 (기본값: "geometry_type")
    - coords_col: 좌표 정보를 포함한 컬럼 이름 (기본값: "coordinates")
    - crs: 좌표계 (기본값: EPSG:4326)
    
    Returns:
    - GeoDataFrame
    """
    # 문자열로 저장된 coordinates를 리스트로 변환
    df[coords_col] = df[coords_col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Shapely의 shape 함수로 geometry 생성
    df["geometry"] = df.apply(lambda row: shape({
        "type": row[geom_type_col],
        "coordinates": row[coords_col]
    }), axis=1)
    
    # GeoDataFrame 생성
    gdf = gpd.GeoDataFrame(df, geometry="geometry")
    gdf.set_crs(crs, inplace=True)  # 좌표계 설정
    return gdf

# 사용 예제
gdf = convert_to_geodataframe_from_columns(df['sidewalk'], geom_type_col="geometry_type", coords_col="coordinates")
print(gdf.head())


                                UFID  WIDT    QUAL    BYYN    KIND      SCLS  \
0  100037802013A003112094078ca764490   0.0  SWQ000  BYC002  SWK001  A0033324   
1  100037802013A0031b7bc7ce581b84138   0.0  SWQ000  BYC002  SWK001  A0033324   
2  100037802077A0031d60630b22db0486f   0.0  SWQ000  BYC002  SWK001  A0033324   
3  100037802096A003105f4d682956140ed   0.0  SWQ000  BYC002  SWK001  A0033324   
4  100037802096A00311bc1074458ef4efc   0.0  SWQ000  BYC002  SWK001  A0033324   

       FMTA    geometry_type  \
0  S2113478  MultiLineString   
1  S2113478  MultiLineString   
2  S2113478  MultiLineString   
3  S2113478  MultiLineString   
4  S2113478  MultiLineString   

                                         coordinates  \
0  [[[128.31811208200418, 37.96487488673088], [12...   
1  [[[128.31799061665288, 37.96469399605625], [12...   
2  [[[128.41457352143004, 37.82164708110149], [12...   
3  [[[128.38489984062628, 37.75901624350924], [12...   
4  [[[128.38478630618363, 37.76143740194771], 

In [17]:
output_folder = "./preprocessed_geojson_data"
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

# GeoDataFrame 저장
geojson_file = os.path.join(output_folder, "sidewalk.geojson")  # 저장할 파일 이름
gdf.to_file(geojson_file, driver="GeoJSON")  # GeoJSON 형식으로 저장

print(f"GeoJSON 파일이 저장되었습니다: {geojson_file}")

GeoJSON 파일이 저장되었습니다: ./preprocessed_geojson_data\sidewalk.geojson
